In [1]:
from src.WikidataTextifier import WikidataEntity

earth = WikidataEntity.from_id("Q64", external_ids=True, lang='de')
earth.to_json()

{'QID': 'Q64',
 'label': 'Berlin',
 'description': 'Hauptstadt und bevölkerungsreichste Stadt sowie Land der Bundesrepublik Deutschland',
 'aliases': ['BE',
  'Berlin (Deutschland)',
  'Land Berlin',
  'Bundeshauptstadt Berlin',
  'DE-BE',
  'Bln',
  'Stadt Berlin',
  'Berlin, Deutschland',
  'Bln.'],
 'claims': [{'PID': 'P610',
   'property_label': 'höchster Punkt',
   'datatype': 'wikibase-item',
   'values': [{'value': {'QID': 'Q19259618', 'label': 'Arkenberge'},
     'qualifiers': [{'PID': 'P2044',
       'property_label': 'Höhe über dem Meeresspiegel',
       'datatype': 'quantity',
       'values': [{'value': {'amount': '+121.9',
          'unit': 'Meter',
          'unit_QID': 'Q11573'}}]}]}]},
  {'PID': 'P1151',
   'property_label': 'Themenportal',
   'datatype': 'wikibase-item',
   'values': [{'value': {'QID': 'Q3248436', 'label': 'Portal:Berlin'}}]},
  {'PID': 'P31',
   'property_label': 'ist ein(e)',
   'datatype': 'wikibase-item',
   'values': [{'value': {'QID': 'Q1901835',